### Final Team Project: Multi-Agent Financial Analysis System (Preliminary) Draft :-   

In [1]:
# !pip install -U langchain-community

In [2]:
# # ! pip install groq
# # ! pip install langchain_groq
# # ! pip install tools
# ! pip install yfinance pandas pandas_ta

#### 1. Performing Imports and Setting up LLM :-   

In [1]:
import os
import time
import torch
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as ta
from datetime import datetime, timedelta

from langchain.tools import Tool
from langchain import LLMChain
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import ZeroShotAgent, AgentExecutor
from langchain.document_loaders.web_base import WebBaseLoader
from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool



In [2]:
GROQ_API_Key = "<YOUR API Key>"
SERPER_API_Key = "<YOUR API Key>"

# Set up Serper API Key as Environmental variable
# Set up GROQ API Key as Environmental variable
os.environ["SERPER_API_KEY"] = SERPER_API_Key
os.environ["GROQ_API_KEY"] = GROQ_API_Key

In [3]:
llm = ChatGroq(
    model = "llama-3.1-8b-instant",
    api_key = os.environ["GROQ_API_KEY"],
    max_tokens = 256, # Maximum number of that can be generated by the model over a single run
    temperature = 0.1 # To ensure precise response. It indicates randomness of token generation where a lower value results in more deterministic token generation.
)

#### 2. Defining Specialized Agent Functions :-    

In [4]:
def format_results(docs, query):
  title_content_list = []
  # Iterate through the loaded web pages and format their respective titles and content
  for doc in docs:
    title = doc.metadata.get('title','No title available')
    page_content = doc.page_content.strip() if query in doc.page_content else "" # Removing unnecessary training/leading spaces
    title_content = f"{title}:{page_content}\n"
    title_content_list.append(title_content)
  # Join formatted content and titles into a single string
  return "\n".join(title_content_list)

In [5]:
# Function to retrieve Stock symbol and stock related news from Yahoo Finance
def financial_news(ticker, n_search_results=2):
  # Retrieve the news from Yahoo Finance
  links = []
  try:
    # Create a yfinance.Ticker object for the specified ticker
    company = yf.Ticker(ticker)
    # Retrieving news articles of type "STORY" and storing their respective links
    links = [n["link"] for n in company.news if n["type"] == "STORY"]
    print(f"Links are retrieved and stored successfully")
  except:
    print(f"No news found from Yahoo Finance")

  # Create a WebBaseLoader to load web pages using the collected links
  loader = WebBaseLoader(links)
  docs = loader.load()
  # Formatting the results by combining titles and page content
  data = format_results(docs, ticker)
  print(f"Completed retrieving news for {ticker}")
  return data






##### Stock Financial Statements :    

We are defining a function script that will make use of yfinance library to gain access of financial data for a given stock such as NVIDIA (ticker symbol : NVDA). The data is inclusive of key stock price information and financial statements.

In [6]:
def financial_statements(ticker):
  # Creating a Ticker object for the specified stock ticker
  company = yf.Ticker(ticker)

  # Modules to fetch company's balance sheet, cash flow and income statement data
  balance_sheet_statement = company.balance_sheet
  cash_flow_statement = company.cash_flow
  income_statement = company.income_stmt

  # Set up the file name with csv pre-fix
  csv_file_prefix = f"{ticker}_financial_"

  # Retrieve the Stock price data
  stock_data = yf.download(ticker, period='1y', interval='1d')

  # Saving stock price data to the CSV file
  data_csv_filename = csv_file_prefix + "stock_data.csv"
  stock_data.to_csv(data_csv_filename)

  # Save financial statements to the CSV file
  balance_sheet_statement_csv_filename = csv_file_prefix + "balance_sheet_statement.csv"
  cash_flow_statement_csv_filename = csv_file_prefix + "cash_flow_statement.csv"
  income_statement_csv_filename = csv_file_prefix + "income_statement.csv"

  # Save the data from balance sheet, cash flow, and income statement
  balance_sheet_statement.to_csv(balance_sheet_statement_csv_filename)
  cash_flow_statement.to_csv(cash_flow_statement_csv_filename)
  income_statement.to_csv(income_statement_csv_filename)

  print(f"Financial Statements and Stock Price data are saved in respective CSV files")
  return data_csv_filename, balance_sheet_statement_csv_filename, cash_flow_statement_csv_filename, income_statement_csv_filename


#### 3. Configuring and Deploying Agents :-   

##### Tool Components : Configuring a tool list

In [7]:
tools = [
    Tool(
        name = "News Analyst",
        func = financial_news,
        description = "Useful when you want to obtain information about current financial events."
    ),
    Tool(
        name = "Financial Analyst",
        func = financial_statements,
        description = "Useful when you need to analyze the company's stock price and financial statements to find more insight. Input should be company ticker such as TSLA for Tesla, NVDA for NVIDIA."
    )
]

##### Setting up LLM Chain :   

In [8]:
# Setting up a basic LLM chain - This will enable us to create a simple conversational agent, one that is capable of providing investment advice.
template = """
<<SYS>>
As a stock analyst, your role is to provide investment insights based on the company's current financial performance and market trends.
You refrain from providing direct 'Buy' or 'Sell' recommendations to comply with legal regulations.
Utilize the data in your database to create a detailed investment thesis to address the user's request.
Please back your assertions with substantial data and analysis.

Use the following format:
- Insights from News: share the insights found from the recent news. Be sure to mention the specific time frame or period you are analyzing.
- Balance Sheet Analysis: provide an analysis of the company's balance sheet. Be sure to mention the specific time frame or period you are analyzing.
- Income Statement Analysis: write an analysis of its income statement to answer user query. Be sure to mention the specific time frame or period you are analyzing.
-Summary: the final answer to the original input question backed up by the insights and analysis above. Be sure to mention the specific time frame or period you are analyzing.

Note: Anything else relevant to the ticker. You should also encourage the user to conduct further research and consider various factors before making any investment decisions.

Begin!
<</SYS>>

[INST]
{input}
{agent_scratchpad}
[/INST]
"""

In [9]:
# Creating a Prompt Template - Blueprint representing how AI's prompts will be structured
prompt = PromptTemplate(input_variables = ["input","agent_scratchpad"], template = template)

# Creating an LLMChain instance
# It is essentially the chain that connects out language model (llm) with the prompt structure defined above
llm_chain = LLMChain(
    llm = llm,
    prompt = prompt
)

/tmp/ipython-input-2889387134.py:6: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


##### Running and Setting Up the AI Agent :-

In [10]:
# Extracting tool names defined earlier
tool_names = [tool.name for tool in tools]

# Creating an instance of the ZeroShotAgent
# The agent utilizes the LLM Chain alongwith access to pre-defined tools
agent = ZeroShotAgent(llm_chain = llm_chain, allowed_tools = tool_names)

# Setting up Agent Executor
# Repeatedly call the agent with the tools until a final answer is obtained

agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, max_iterations=3, handle_parsing_errors=True)

/tmp/ipython-input-2557658414.py:6: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = ZeroShotAgent(llm_chain = llm_chain, allowed_tools = tool_names)


In [11]:
# Defining a function that will interact with the AI Model
def Agent_operations (ticker):
  try:
    # Convert ticker to upper case
    ticker = ticker.upper()
    # Run agent executor with the input ticker
    response = agent_executor.run(input=ticker)
  except ValueError as e:
    # In the event of ValueError : convert error to String
    response = str(e)
    if not response.startswith('Could not parse LLM Output :'):
      raise e
    response = response.removeprefix("Could not parse LLM Output: ").removesuffix(" ")

    # Return the ticker and response from the agent
    return{
        'input': ticker,
        'text': response
    }

In [12]:
ticker = "APPL"
print(Agent_operations(ticker))


/tmp/ipython-input-2873356656.py:7: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_executor.run(input=ticker)




> Entering new AgentExecutor chain...
**Insights from News:**

For Apple Inc. (APPL), I analyzed recent news from the past 6 months (June 2023 to September 2023). Here are some key insights:

- **Strong Earnings Report**: In July 2023, Apple reported a 10% increase in revenue, driven by strong sales of its iPhone and Mac products. This indicates a positive trend in the company's financial performance.
- **Growth in Services Segment**: Apple's services segment, which includes Apple Music, Apple TV+, and Apple Arcade, has been growing rapidly. In the latest quarter, services revenue increased by 12% year-over-year, contributing to the company's overall growth.
- **Investment in Emerging Technologies**: Apple has been investing heavily in emerging technologies such as artificial intelligence, augmented reality, and 5G. This strategic move is expected to drive growth in the long term.
- **Supply Chain Disruptions**: However, Apple has faced supply chain disruptions due to the ongoing glo